# Basic RNN
- Objective: to understand basics of RNN & LSTM

## Recurrent Neural Networks
- Feedforward neural networks (e.g. MLPs and CNNs) are powerful, but they are not optimized to handle "sequential" data
- In other words, they do not possess "memory" of previous inputs
- For instance, consider the case of translating a corpus. You need to consider the **"context"** to guess the next word to come forward

<img src="http://2.bp.blogspot.com/-9GIdV292xV4/UwOIy6B6koI/AAAAAAAAHi4/X6UGlyHI-_U/s1600/tumblr_ms5qzpFY671r9nm7io1_500.gif" style="width: 500px"/>

<br>
- RNNs are suitable for dealing with sequential format data since they have **"recurrent"** structure
- To put it differently, they keep the **"memory"** of earlier inputs in the sequence
</br>
<img src="http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg" style="width: 600px"/>

<br>
- However, in order to reduce the number of parameters, every layer of different time steps shares same parameters
</br>

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" style="width: 600px"/>

## Load Dataset

In [1]:
import numpy as np

from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

In [3]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

2113536/2110848 [==============================] - 0s 0us/step


In [4]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

In [5]:
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

In [6]:
y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)

In [7]:
y_train = y_data[:1395]
y_test = y_data[1395:]

In [8]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1395, 49, 1)
(599, 49, 1)
(1395, 46)
(599, 46)


## 1. Vanilla RNN
- Vanilla RNNs have a simple structure
- However, they suffer from the problem of "long-term dependencies"
- Hence, they are not able to keep the **sequential memory" for long

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png" style="width: 600px"/>

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [10]:
def vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [11]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 200, batch_size = 50, verbose = 1)

In [12]:
model.fit(X_train, y_train)

Epoch 1/200
28/28 [==============================] - 0s 8ms/step - loss: 3.1467 - accuracy: 0.2158
Epoch 2/200
28/28 [==============================] - 0s 9ms/step - loss: 1.5311 - accuracy: 0.6789
Epoch 3/200
28/28 [==============================] - 0s 8ms/step - loss: 1.2370 - accuracy: 0.7054
Epoch 4/200
28/28 [==============================] - 0s 8ms/step - loss: 1.1850 - accuracy: 0.7147
Epoch 5/200
28/28 [==============================] - 0s 8ms/step - loss: 1.1672 - accuracy: 0.7147
Epoch 6/200
28/28 [==============================] - 0s 9ms/step - loss: 1.1637 - accuracy: 0.7147
Epoch 7/200
28/28 [==============================] - 0s 8ms/step - loss: 1.1581 - accuracy: 0.7147
Epoch 8/200
28/28 [==============================] - 0s 8ms/step - loss: 1.1537 - accuracy: 0.7147
Epoch 9/200
28/28 [==============================] - 0s 9ms/step - loss: 1.1519 - accuracy: 0.7147
Epoch 10/200
28/28 [==============================] - 0s 8ms/step - loss: 1.1513 - accuracy: 0.7147
Epoch 11/

In [13]:
y_pred = model.predict(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
12/12 [==============================] - 0s 3ms/step


In [14]:
y_test_ = np.argmax(y_test, axis = 1)

In [15]:
print(accuracy_score(y_pred, y_test_))

0.7495826377295493


## 2. Stacked Vanilla RNN
- RNN layers can be stacked to form a deeper network

<img src="https://lh6.googleusercontent.com/rC1DSgjlmobtRxMPFi14hkMdDqSkEkuOX7EW_QrLFSymjasIM95Za2Wf-VwSC1Tq1sjJlOPLJ92q7PTKJh2hjBoXQawM6MQC27east67GFDklTalljlt0cFLZnPMdhp8erzO" style="width: 500px"/>

In [16]:
def stacked_vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = True))   # return_sequences parameter has to be set True to stack
    model.add(SimpleRNN(50, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [17]:
model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 200, batch_size = 50, verbose = 1)

In [18]:
model.fit(X_train, y_train)

Epoch 1/200
28/28 [==============================] - 1s 19ms/step - loss: 2.4446 - accuracy: 0.5333
Epoch 2/200
28/28 [==============================] - 1s 20ms/step - loss: 1.2726 - accuracy: 0.7147
Epoch 3/200
28/28 [==============================] - 1s 19ms/step - loss: 1.1797 - accuracy: 0.7147
Epoch 4/200
28/28 [==============================] - 1s 19ms/step - loss: 1.1434 - accuracy: 0.7147
Epoch 5/200
28/28 [==============================] - 1s 18ms/step - loss: 1.1186 - accuracy: 0.7147
Epoch 6/200
28/28 [==============================] - 1s 19ms/step - loss: 1.0987 - accuracy: 0.7147
Epoch 7/200
28/28 [==============================] - 1s 19ms/step - loss: 1.0930 - accuracy: 0.7147
Epoch 8/200
28/28 [==============================] - 1s 19ms/step - loss: 1.0803 - accuracy: 0.7147
Epoch 9/200
28/28 [==============================] - 1s 19ms/step - loss: 1.0779 - accuracy: 0.7154
Epoch 10/200
28/28 [==============================] - 1s 19ms/step - loss: 1.0730 - accuracy: 0.7147

In [19]:
y_pred = model.predict(X_test)

12/12 [==============================] - 0s 6ms/step


In [20]:
print(accuracy_score(y_pred, y_test_))

0.7746243739565943


## 3. LSTM
- LSTM (long short-term memory) is an improved structure to solve the problem of long-term dependencies

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" style="width: 600px"/>

In [21]:
from tensorflow.keras.layers import LSTM

In [22]:
def lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (49,1), return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [23]:
model = KerasClassifier(build_fn = lstm, epochs = 200, batch_size = 50, verbose = 1)

In [24]:
model.fit(X_train, y_train)

Epoch 1/200
28/28 [==============================] - 1s 21ms/step - loss: 3.2411 - accuracy: 0.6029
Epoch 2/200
28/28 [==============================] - 1s 22ms/step - loss: 1.4156 - accuracy: 0.7147
Epoch 3/200
28/28 [==============================] - 1s 22ms/step - loss: 1.1957 - accuracy: 0.7147
Epoch 4/200
28/28 [==============================] - 1s 23ms/step - loss: 1.1572 - accuracy: 0.7147
Epoch 5/200
28/28 [==============================] - 1s 22ms/step - loss: 1.1355 - accuracy: 0.7147
Epoch 6/200
28/28 [==============================] - 1s 22ms/step - loss: 1.1212 - accuracy: 0.7147
Epoch 7/200
28/28 [==============================] - 1s 22ms/step - loss: 1.1070 - accuracy: 0.7147
Epoch 8/200
28/28 [==============================] - 1s 22ms/step - loss: 1.0964 - accuracy: 0.7147
Epoch 9/200
28/28 [==============================] - 1s 22ms/step - loss: 1.0866 - accuracy: 0.7147
Epoch 10/200
28/28 [==============================] - 1s 22ms/step - loss: 1.0800 - accuracy: 0.7147

In [25]:
y_pred = model.predict(X_test)

12/12 [==============================] - 0s 7ms/step


In [26]:
# accuracy improves by adopting LSTM structure
print(accuracy_score(y_pred, y_test_))

0.8464106844741235


## 4. Stacked LSTM
- LSTM layers can be stacked as well

In [27]:
def stacked_lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (49,1), return_sequences = True))
    model.add(LSTM(50, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [28]:
model = KerasClassifier(build_fn = stacked_lstm, epochs = 200, batch_size = 50, verbose = 1)

In [29]:
model.fit(X_train, y_train)

Epoch 1/200
28/28 [==============================] - 1s 49ms/step - loss: 2.6792 - accuracy: 0.6437
Epoch 2/200
28/28 [==============================] - 1s 47ms/step - loss: 1.2435 - accuracy: 0.7147
Epoch 3/200
28/28 [==============================] - 1s 46ms/step - loss: 1.1688 - accuracy: 0.7147
Epoch 4/200
28/28 [==============================] - 1s 47ms/step - loss: 1.1525 - accuracy: 0.7147
Epoch 5/200
28/28 [==============================] - 1s 48ms/step - loss: 1.1345 - accuracy: 0.7147
Epoch 6/200
28/28 [==============================] - 1s 48ms/step - loss: 1.1144 - accuracy: 0.7147
Epoch 7/200
28/28 [==============================] - 1s 47ms/step - loss: 1.1124 - accuracy: 0.7147
Epoch 8/200
28/28 [==============================] - 1s 47ms/step - loss: 1.0941 - accuracy: 0.7147
Epoch 9/200
28/28 [==============================] - 1s 48ms/step - loss: 1.0806 - accuracy: 0.7147
Epoch 10/200
28/28 [==============================] - 1s 47ms/step - loss: 1.0536 - accuracy: 0.7147

In [30]:
y_pred = model.predict(X_test)

12/12 [==============================] - 0s 13ms/step


In [31]:
print(accuracy_score(y_pred, y_test_))

0.8480801335559266
